In [3]:
!pip install tweepy
import tweepy
import json
import time
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAABoebgEAAAAA8nOwpOH6GviG0pVFbGAEowC6lrE%3DvQmBGiPwQ2EJKcoMQUNYDwgb0d5vkjxvSPtkuqY0FFGY9D2yS8")


    
    

     |████████████████████████████████| 77 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 2.9 MB/s eta 0:00:01
  Attempting uninstall: oauthlib
    Found existing installation: oauthlib 3.1.1
    Uninstalling oauthlib-3.1.1:
      Successfully uninstalled oauthlib-3.1.1


In [48]:
import os
import pandas as pd
import json
import glob
from itertools import chain
import math
import numpy as np
file = "test.data.txt"
file_open = open('./project-data/' + file, 'r')
count = 0
delta = pd.DataFrame()
for line in file_open.readlines():
    tweet_data={}
    line = line.strip()
    each_tweet = line.split(',')
    main_tweet_id = each_tweet[0]
    main_tweet = client.get_tweet(main_tweet_id)
    each_tweet = each_tweet[1:]
    # count+=len(each_tweet)
    if len(each_tweet) > 100:
        num_sub_list = math.ceil(len(each_tweet)/100)
        a = np.array_split(each_tweet, num_sub_list)
        tweets = []
        for sub_list in a:
            sub_list = sub_list.tolist()
            tw = client.get_tweets(sub_list, tweet_fields=["created_at"])
            tweets.append(tw.data)
        
        tweets =  list(chain.from_iterable(tweets))

        # tweets = client.get_tweets()
    elif len(each_tweet) <= 100 and len(each_tweet) > 0:
        tweets = client.get_tweets(each_tweet,tweet_fields=["created_at"])
        tweets = tweets.data if tweets.data is not None else []
    replies = []
    print(main_tweet_id)
    if len(tweets) > 0:
        for tweet in tweets:
            reply_data = {}
            reply_data['tweet_id'] = tweet.id
            reply_data['tweet'] = tweet.text
            reply_data['created_at'] = tweet.created_at.timestamp()
            replies.append(reply_data)

    if main_tweet.data is not None:
        tweet_data['main_tweet'] = main_tweet.data.text
        tweet_data['main_tweet_id'] = main_tweet.data.id
    else:
        tweet_data['main_tweet'] = ""
        tweet_data['main_tweet_id'] = main_tweet_id
    tweet_data['replies'] = json.dumps(replies) 
    
    delta = delta.append(pd.DataFrame([tweet_data]))
    delta.to_csv('./project-data/%s.csv'%file, index=False)
    time.sleep(10)
 
    

file_open.close()


1250219300389974016
554886875303780352
1237901309011021825
524958128392376320
1239295488677085185
1250219998842216448
1241047451483553792
1221830244895686658
489975398151618560
544289941996326912
676870737932742656
520244836084228096
626770498328895488
538408710084001793
1255794736759140352
672632899921833984
1246818494995447809
1237315523773333504
525291306000846848
1248595704240619523
1235494521753997312
524925987239120897
387030572779847680
407169212428206080
1236192572575485952
1253621063415672833
407212665350942720
514495841600811009
1255068286141829120
1239374019633217536
1254538699414609920
1250154897439436806
1250220791544705025
1242485242046205955
1253157289587929089
1235014618910728196
1226224930548834306
616421546702336000
510931521306243072
544350712365207552
1249603458132594689
1252780892910751744
507242819623587841
1275667941237960705
1235493800426037248
1270775529319231489
519965814175567872
632377165477191680
435998882250899456
1239811802289958913
506656271517622272
123

In [191]:
# delta = pd.read_csv('./project-data/%s.csv'%file,keep_default_na=False)
label = open('./project-data/train.label.txt', 'r')
label_data = label.readlines()
# delta
# delta = delta.iloc[:, 1:]
labels_convert = {
    "rumour": 0,
    "nonrumour": 1
}

label_data = [labels_convert[x.strip()] for x in label_data]
delta["main_tweet"] = delta["main_tweet"].astype(str)
delta['main_tweet'].replace('nan', '', inplace=True)
delta['label'] = label_data

delta.to_csv('./project-data/%s.csv'%file, index=False)


# Prepocessing

In [193]:
# delta[delta['main_tweet']==NaN].index
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')

train_data = delta.copy()
train_data['main_tweet'] = train_data['main_tweet'].fillna('')
train_data.replace('', np.nan, inplace=True)
train_data.dropna(subset=['main_tweet'], inplace=True)
train_data
# def tokenize(text):
#     tokens = nltk.word_tokenize(text)
#     stems = []
#     for item in tokens:
#         if item not in stopwords:
#             stems.append(PorterStemmer().stem(item))
#     return stems
# #lower case all
# train_data['main_tweet'] = train_data['main_tweet'].apply(lambda x: x.lower())
# train_data['main_tweet'] = train_data['main_tweet'].apply(lambda x: x.replace('\n', ' '))
# train_data['main_tweet'] = train_data['main_tweet'].apply(lambda x: tokenize(x))
# train_data

[nltk_data] Downloading package punkt to /home/kan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,main_tweet,main_tweet_id,replies,label
0,5. Can regularly rinsing your nose with saline...,1250219300389974016,"[{""tweet_id"": 1250219116993974272, ""tweet"": ""4...",1
1,French police chief killed himself after #Char...,554886875303780352,"[{""tweet_id"": 554959644125167617, ""tweet"": ""De...",0
2,Coronavirus disease (COVID-19) advice for the ...,1237901309011021825,"[{""tweet_id"": 1237901311439450112, ""tweet"": ""I...",1
3,Ottawa police confirm that there were multiple...,524958128392376320,"[{""tweet_id"": 524961934064754688, ""tweet"": ""@W...",1
4,if the primary focus of a government isn't to ...,1239295488677085185,[],1
...,...,...,...,...
1889,4. It cannot be transmitted through goods manu...,1237545128828342277,"[{""tweet_id"": 1237545126278258703, ""tweet"": ""#...",1
1890,Desperate Ted Cruz Claims Planned Parenthood S...,671181758692507648,"[{""tweet_id"": 671200376843067392, ""tweet"": ""@B...",0
1891,"""Thoughts and prayers are not enough."" Pres. O...",672513234419638273,"[{""tweet_id"": 672513853645717504, ""tweet"": ""@A...",0
1892,Police have surrounded this building where the...,553508098825261056,"[{""tweet_id"": 553509546602553344, ""tweet"": ""@N...",1
